In [8]:
import cv2
import numpy as np

img = cv2.imread('/home/hp/Documents/Daily_Task/Day_2/uv_nov11/5296_uv.png')
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

lower_blue = np.array([100, 50, 50])
upper_blue = np.array([130, 255, 255])

mask = cv2.inRange(hsv, lower_blue, upper_blue)

In [9]:
kernel = np.ones((6, 6), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

In [10]:
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [15]:
if contours:
    largest_contour = max(contours, key=cv2.contourArea)
    
    x, y, w, h = cv2.boundingRect(largest_contour)
    (cx, cy), radius = cv2.minEnclosingCircle(largest_contour)
    
    # Add padding to increase bbox size
    padding = 20
    x_bbox = max(0, x - padding)
    y_bbox = max(0, y - padding)
    w_bbox = w + 2 * padding
    h_bbox = h + 2 * padding
    
    # Ensure bbox doesn't exceed image boundaries
    w_bbox = min(w_bbox, img.shape[1] - x_bbox)
    h_bbox = min(h_bbox, img.shape[0] - y_bbox)
    
    # Create mask using actual contour shape (not circle)
    yarn_mask = np.zeros_like(mask)
    cv2.drawContours(yarn_mask, [largest_contour], -1, 255, -1)
    
    # Crop the yarn cone region
    yarn_cone_mask = yarn_mask[y_bbox:y_bbox+h_bbox, x_bbox:x_bbox+w_bbox]
    yarn_cone_img = img[y_bbox:y_bbox+h_bbox, x_bbox:x_bbox+w_bbox]
    
    # Apply mask to show only yarn cone
    yarn_cone_isolated = cv2.bitwise_and(yarn_cone_img, yarn_cone_img, mask=yarn_cone_mask)
    
    cv2.rectangle(img, (x_bbox, y_bbox), (x_bbox+w_bbox, y_bbox+h_bbox), (0, 255, 0), 2)
    cv2.putText(img, 'Yarn Cone', (x_bbox, y_bbox-10), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

cv2.namedWindow('Original', cv2.WINDOW_NORMAL)
cv2.namedWindow('Yarn Cone Mask', cv2.WINDOW_NORMAL)
cv2.namedWindow('Isolated Yarn Cone', cv2.WINDOW_NORMAL)

cv2.imshow('Original', img)
cv2.imshow('Yarn Cone Mask', yarn_cone_mask)
cv2.imshow('Isolated Yarn Cone', yarn_cone_isolated)

cv2.imwrite('Yarn_mask.png', yarn_cone_mask)
cv2.imwrite('yarn.png', yarn_cone_isolated)

cv2.waitKey(0)
cv2.destroyAllWindows()